Example of the process of doing DEMs for all orbits for a given observation of an AR – where we don't window out any intervals because of shifts, etc. Instead, we will save energy-specific information about the percent of NuSTAR emission in the chosen region as a function of time, for later inspection. This will help identify cases where pointing shifts might cause unphysical distortion to the DEM inputs. 

Note: auto-download of an AIA file (for use in the initial co-alignmnet) will break while the JSOC is still down. We will add a thing where you can point to an existing AIA file instead.

Overview:

- Define orbits
- Run time interval selection
- Examine resulting intervals
- Manually establish a co-alignment shift between NuSTAR and AIA
- Automatically find co-alignment shifts + make regions for all other time intervals (note: this relies on the assumption that the COM is a good representation of the location of the brightest source, i.e. that the NuSTAR data is primarially one blob).
- Save AIA region files for NCCS input
- NOT IN THIS NOTEBOOK: YOU THEN TAKE THOSE AND MAKE AIA INPUTS ON THE NCCS
- Conduct AIA/NuSTAR DEMs as a function of time, given all the above
- Plot results.
- Print some stats about "left out" times.

In [1]:
# import matplotlib.pyplot as plt
# import numpy as np
# import glob
# from astropy.io import fits
# from astropy import units as u
# import importlib
# import pathlib

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

import nustar_dem_prep as nu
# import initial_analysis as ia
# import orbit_auto as oa
import time_interval_selection as tis
import nustar_utilities as nuutil
# import gauss2D as g2d
import all_nu_analysis as ana
import visualize_dem_results as viz
import images_and_coalignment as iac


import pickle
import pathlib
import importlib
import numpy as np
import subprocess


In [30]:
importlib.reload(ana)
key = '19-feb-16'
method='fit'

#===============================================================
#Update dictionary to have correct aia path for NCCS-prepped data.
#===============================================================
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

data[key] = ARDict

with open('all_targets.pickle', 'wb') as f:
         # Pickle the 'data' dictionary using the highest protocol available.
         pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

#===============================================================
#DO DEMS


for id in id_dirs:
    evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

ana.do_key_dem(key, plot_xrt=False, method=method) #missing_last=True, missing_orbit=1, plot_xrt=False)

#===============================================================


#===============================================================
#Copy DEM result files to common directory


minT=5.6
maxT=7.2

if method == 'fit':
    all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                missing_last=False)

    for time in all_time_intervals_list:
        timestring = time[0].strftime('%H-%M-%S')
        stopstring = time[1].strftime('%H-%M-%S')
        timestring=timestring+'_'+stopstring
    
        demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
        print(demfile)

        status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 


if method in ['input', 'double']:
    if method=='input':
        region_dirs = iac.find_region_dirs(working_dir)
    if method=='double':
        gauss_stats = ARDict['gauss_stats']
        sep_axis = gauss_stats[0][0]
        region_dirs = iac.find_direction_dirs(working_dir, sep_axis)
        #print(region_dirs)

    all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True) 

    for i in range(0, len(region_dirs)):
        atl = all_all_time_intervals_list[i]
        r = region_dirs[i]

        for time in atl:
            timestring = time[0].strftime('%H-%M-%S')
            stopstring = time[1].strftime('%H-%M-%S')
            timestring=timestring+'_'+stopstring
        
            demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
            demfile_ = demfile.split('/')[-1]+'_region_'+str(i)+'.pickle'
            print(demfile_)
    
            status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 

#===============================================================



18-54-46 19-54-54
20-31-29 21-22-26
22-08-12 23-03-57
/Users/jmdunca2/do-dem/initial_dem_19feb16/all_aia_dicts_19-feb-16_post//orbit_20102011001/
filename:  /Users/jmdunca2/do-dem/initial_dem_19feb16/all_aia_dicts_19-feb-16_post//orbit_20102011001//18-54-50_19-08-55_aia_prep.pickle
Start Time:  2016-02-19 18:54:50
Stop Time:  2016-02-19 19:08:55

No  Be_thin  files.
No  Be_med  files.
No  Al_poly  files.
No XRT files found, exiting.
Something is preventing use of XRT.


Using grades 0-4 NuSTAR events.
ARF File:  ['/Users/jmdunca2/do-dem/initial_dem_19feb16/18-54-50_19-08-55/nu20102011001A06_0_4_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem/initial_dem_19feb16/18-54-50_19-08-55/nu20102011001A06_0_4_p_sr.rmf']
PHA File:  ['/Users/jmdunca2/do-dem/initial_dem_19feb16/18-54-50_19-08-55/nu20102011001A06_0_4_p_sr.pha']
ARF File:  ['/Users/jmdunca2/do-dem/initial_dem_19feb16/18-54-50_19-08-55/nu20102011001A06_21_24_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem/initial_dem_19feb16/18-54-50_19

In [29]:
import glob
o=2
prepped_aia_dir=ARDict['prepped_aia']
print(prepped_aia_dir)
orbit_aia_dir = prepped_aia_dir+'/orbit_'+obsids[o]+'/'
print(orbit_aia_dir)
files = glob.glob(orbit_aia_dir+'/*')

for filename in files: 
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    #print(data)
    newdata = data['region0']

    with open(filename, 'wb') as f:
        pickle.dump(newdata, f, pickle.HIGHEST_PROTOCOL)

/Users/jmdunca2/do-dem/initial_dem_19feb16/all_aia_dicts_19-feb-16_post/
/Users/jmdunca2/do-dem/initial_dem_19feb16/all_aia_dicts_19-feb-16_post//orbit_20102013001/


In [2]:
def move_dems(key, method):


    with open('all_targets.pickle', 'rb') as f:
        data = pickle.load(f)
    
    ARDict = data[key]
    
    id_dirs = ARDict['datapaths']
    obsids = ARDict['obsids']
    working_dir = ARDict['working_dir']
    print(working_dir)

    minT=5.6
    maxT=7.2
    
    if method == 'fit':
        all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                    missing_last=False)
    
        for time in all_time_intervals_list:
            timestring = time[0].strftime('%H-%M-%S')
            stopstring = time[1].strftime('%H-%M-%S')
            timestring=timestring+'_'+stopstring
        
            demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
            print(demfile)
    
            status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 
    
    
    if method in ['input', 'double']:
        if method=='input':
            region_dirs = iac.find_region_dirs(working_dir)
        if method=='double':
            gauss_stats = ARDict['gauss_stats']
            sep_axis = gauss_stats[0][0]
            region_dirs = iac.find_direction_dirs(working_dir, sep_axis)
            #print(region_dirs)
    
        all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True) 
    
        for i in range(0, len(region_dirs)):
            atl = all_all_time_intervals_list[i]
            r = region_dirs[i]
    
            for time in atl:
                timestring = time[0].strftime('%H-%M-%S')
                stopstring = time[1].strftime('%H-%M-%S')
                timestring=timestring+'_'+stopstring
            
                demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
                demfile_ = demfile.split('/')[-1]+'_region_'+str(i)+'.pickle'
                print(demfile_)
        
                status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 
        
 

fitkeys = ['11-sep-17',
             '29-may-18_1', '09-sep-18', '10-sep-18', 
        '12-apr-19', '13-apr-19', 
        '06-jun-20', '07-jun-20', 
        '08-jun-20', '09-jun-20',
        '29-apr-21', '03-may-21_1', '03-may-21_2','30-jul-21_2']

inputkeys = ['01-sep-15', '02-sep-15',
            '26-jul-16_1', '27-jul-16_1', '26-jul-16_2',
             '12-sep-17', '13-sep-17', '10-oct-17'
            ]

doublekeys = ['20-jul-21', '30-jul-21_1']

# for k in fitkeys:
#     move_dems(k, 'fit')

# for k in inputkeys:
#     move_dems(k, 'input')

# for k in doublekeys:
#     move_dems(k, 'double')

In [3]:
move_dems('30-jul-21_2', 'fit')

/Users/jmdunca2/do-dem/initial_dem_30jul21_2/
/Users/jmdunca2/do-dem/initial_dem_30jul21_2/19-41-40_20-16-55/19-41-40_20-16-55_5.6_7.2_30-jul-21_2_MC_DEM_result.pickle
/Users/jmdunca2/do-dem/initial_dem_30jul21_2/20-16-55_20-42-10/20-16-55_20-42-10_5.6_7.2_30-jul-21_2_MC_DEM_result.pickle


In [7]:
demfile_

'04-41-50_05-41-25_5.6_7.2_10-oct-17_MC_DEM_result.pickle_region_1.pickle'

In [5]:
# importlib.reload(ana)
# key = '30-jul-21_1'
# method='double'

# #===============================================================
# #Update dictionary to have correct aia path for NCCS-prepped data.
# #===============================================================
# with open('all_targets.pickle', 'rb') as f:
#     data = pickle.load(f)

# ARDict = data[key]

# id_dirs = ARDict['datapaths']
# obsids = ARDict['obsids']
# working_dir = ARDict['working_dir']

# ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

# data[key] = ARDict

# with open('all_targets.pickle', 'wb') as f:
#          # Pickle the 'data' dictionary using the highest protocol available.
#          pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

# #===============================================================
# #DO DEMS

# import initial_analysis as ia
# import nustar_utilities as nuutil

# for id in id_dirs:
#     evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
#     time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
#     timerange = [time0, time1]
#     print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

# ana.do_key_dem(key, plot_xrt=False, method=method) #missing_last=True, missing_orbit=1, plot_xrt=False)

# #===============================================================


# #===============================================================
# #Copy DEM result files to common directory


# minT=5.6
# maxT=7.2

# if method == 'fit':
#     all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
#                                                                 missing_last=False)
    
#     for time in all_time_intervals_list:
#         timestring = time[0].strftime('%H-%M-%S')
#         stopstring = time[1].strftime('%H-%M-%S')
#         timestring=timestring+'_'+stopstring
        
#         demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
#         print(demfile)
    
#         status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 


# if method in ['input', 'double']:
#     if method=='input':
#         region_dirs = iac.find_region_dirs(working_dir)
#     if method=='double':
#         gauss_stats = ARDict['gauss_stats']
#         sep_axis = gauss_stats[0][0]
#         region_dirs = iac.find_direction_dirs(working_dir, sep_axis)
#         #print(region_dirs)

#     all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True) 

#     for i in range(0, len(region_dirs)):
#         atl = all_all_time_intervals_list[i]
#         r = region_dirs[i]

#         for time in atl:
#             timestring = time[0].strftime('%H-%M-%S')
#             stopstring = time[1].strftime('%H-%M-%S')
#             timestring=timestring+'_'+stopstring
            
#             demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
#             demfile_ = demfile[70:-7]+'_region_'+str(i)+'.pickle'
#             print(demfile_)
        
#             status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 
    
# #===============================================================



In [6]:
importlib.reload(ana)
key = '12-sep-17'
method='input'

#===============================================================
#Update dictionary to have correct aia path for NCCS-prepped data.
#===============================================================
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

data[key] = ARDict

with open('all_targets.pickle', 'wb') as f:
         # Pickle the 'data' dictionary using the highest protocol available.
         pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

#===============================================================
#DO DEMS

import nustar_utilities as nuutil

for id in id_dirs:
    evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

ana.do_key_dem(key, plot_xrt=False, method=method) #missing_last=True, missing_orbit=1, plot_xrt=False)

#===============================================================


#===============================================================
#Copy DEM result files to common directory


minT=5.6
maxT=7.2

if method == 'fit':
    all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                missing_last=False)
    
    for time in all_time_intervals_list:
        timestring = time[0].strftime('%H-%M-%S')
        stopstring = time[1].strftime('%H-%M-%S')
        timestring=timestring+'_'+stopstring
        
        demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
        print(demfile)
    
        status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 


if method == 'input':
    region_dirs = iac.find_region_dirs(working_dir)
    all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True)

    for i in range(0, len(region_dirs)):
        atl = all_all_time_intervals_list[i]
        r = region_dirs[i]

        for time in atl:
            timestring = time[0].strftime('%H-%M-%S')
            stopstring = time[1].strftime('%H-%M-%S')
            timestring=timestring+'_'+stopstring
            
            demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
            demfile_ = demfile[70:-7]+'_region_'+str(i)+'.pickle'
            print(demfile_)
        
            status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 
    
#===============================================================



20-55-57 21-46-17
22-32-38 23-28-23

/Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/ /Users/jmdunca2/do-dem/initial_dem_12sep17/gauss_cen_80310229001_A_user_input_0.reg
Start Time:  2017-09-12 20:56:00
Stop Time:  2017-09-12 20:56:30

No  Be_thin  files.
No  Be_med  files.
No  Al_poly  files.
No XRT files found, exiting.
Something is preventing use of XRT.


Using input regfile:
/Users/jmdunca2/do-dem/initial_dem_12sep17/gauss_cen_80310229001_A_user_input_0.reg
Using grades 0-4 NuSTAR events.
ARF File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/20-56-00_20-56-30/nu80310229001A06_0_4_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/20-56-00_20-56-30/nu80310229001A06_0_4_p_sr.rmf']
PHA File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/20-56-00_20-56-30/nu80310229001A06_0_4_p_sr.pha']
ARF File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/20-56-00_20-56-30/nu80310229001A06_21_24_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem

Traceback (most recent call last):
  File "/Users/jmdunca2/do-dem//dodem/xrt_dem_prep.py", line 367, in load_xrt
    ssw_resp = ssw.run(agr_path, args=inputs)
  File "/Users/jmdunca2/miniforge3/lib/python3.10/site-packages/hissw/environment.py", line 185, in run
    self._check_for_errors(cmd_output, verbose, **kwargs)
  File "/Users/jmdunca2/miniforge3/lib/python3.10/site-packages/hissw/environment.py", line 202, in _check_for_errors
    raise IDLLicenseError(stderr)
hissw.util.IDLLicenseError: IDL 8.9.0 (darwin x86_64 m64).
(c) 2023, L3Harris Geospatial Solutions, Inc.

% Failed to acquire license.
% A license is not available to run this product. Use the license
   administration utility to determine which licenses are available or
   activate an entitlement.



None
Something is wrong with the SSWIDL run - make sure the following IDL script exists:
/Users/jmdunca2/do-dem//hissw_idl/xrt_tresp_hissw_wrapper_aiaspec.pro
Something is preventing use of XRT.


Using input regfile:
/Users/jmdunca2/do-dem/initial_dem_12sep17/gauss_cen_80310229001_A_user_input_1.reg
Using grades 0-4 NuSTAR events.
ARF File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_1/21-27-00_21-32-10/nu80310229001A06_0_4_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_1/21-27-00_21-32-10/nu80310229001A06_0_4_p_sr.rmf']
PHA File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_1/21-27-00_21-32-10/nu80310229001A06_0_4_p_sr.pha']
ARF File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_1/21-27-00_21-32-10/nu80310229001A06_21_24_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_1/21-27-00_21-32-10/nu80310229001A06_21_24_p_sr.rmf']
PHA File:  ['/Users/jmdunca2/do-dem/initial_dem_12sep17/region_1/21-27-00_21-32-10/nu80

cp: /Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/20-59-30_21-00-00/20-59-30_21-00-00_5.6_7.2_12-sep-17_MC_DEM_result.pickle: No such file or directory
cp: /Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/21-00-00_21-00-30/21-00-00_21-00-30_5.6_7.2_12-sep-17_MC_DEM_result.pickle: No such file or directory


21-01-00_21-01-30_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-01-30_21-02-00_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-02-00_21-02-30_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-02-30_21-03-00_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-03-00_21-03-30_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-03-30_21-04-00_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-04-00_21-04-30_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-04-30_21-05-00_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-05-00_21-05-30_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-05-30_21-06-00_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-06-00_21-06-30_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-06-30_21-07-00_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-07-00_21-07-30_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-07-30_21-08-00_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-08-00_21-08-30_5.6_7.2_12-sep-17_MC_DEM_result_region_0.pickle
21-08-30_2

In [ ]:
len('/Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/20-56-30_20-57-00/')

In [ ]:
importlib.reload(ana)
key = '09-sep-18'

#===============================================================
#Update dictionary to have correct aia path for NCCS-prepped data.
#===============================================================
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

data[key] = ARDict

with open('all_targets.pickle', 'wb') as f:
         # Pickle the 'data' dictionary using the highest protocol available.
         pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

#===============================================================
#DO DEMS

import initial_analysis as ia
import nustar_utilities as nuutil

for id in id_dirs:
    evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

ana.do_key_dem(key, missing_last=True, missing_orbit=1, plot_xrt=False)

#===============================================================


#===============================================================
#Copy DEM result files to common directory


minT=5.6
maxT=7.2

all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                missing_last=False)

for time in all_time_intervals_list:
    timestring = time[0].strftime('%H-%M-%S')
    stopstring = time[1].strftime('%H-%M-%S')
    timestring=timestring+'_'+stopstring
    
    demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
    print(demfile)

    status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 

#===============================================================



In [51]:
keys = ['01-sep-15', '02-sep-15',
        '26-jul-16_1', '27-jul-16_1',
        '11-sep-17', '12-sep-17', '13-sep-17',
        '29-may-18_1', '09-sep-18', '10-sep-18', 
        '12-apr-19', '13-apr-19', 
        '06-jun-20', '07-jun-20', 
        '08-jun-20', '09-jun-20',
        '29-apr-21', '03-may-21_1', '03-may-21_2', '20-jul-21', 
        '30-jul-21_1', 
        '30-jul-21_2']

with open('all_targets.pickle', 'rb') as f:
    all_targets = pickle.load(f)
    
done_sources = {k: all_targets[k] for k in keys}
ana.get_exposures(done_sources, dogoes=False)

01-sep-15
Duration:  172.42 min | Effective Exposure:  3.01 min | Livetime %:  0.02
02-sep-15
Duration:  238.83 min | Effective Exposure:  5.27 min | Livetime %:  0.02
26-jul-16_1
Duration:  139.65 min | Effective Exposure:  7.63 min | Livetime %:  0.05
27-jul-16_1
Duration:  60.47 min | Effective Exposure:  2.35 min | Livetime %:  0.04
11-sep-17
Duration:  229.05 min | Effective Exposure:  1.19 min | Livetime %:  0.01
12-sep-17
Duration:  106.08 min | Effective Exposure:  9.84 min | Livetime %:  0.09
13-sep-17
Duration:  119.1 min | Effective Exposure:  13.81 min | Livetime %:  0.12
29-may-18_1
Duration:  263.75 min | Effective Exposure:  4.67 min | Livetime %:  0.02
09-sep-18
Duration:  179.25 min | Effective Exposure:  24.02 min | Livetime %:  0.13
10-sep-18
Duration:  179.27 min | Effective Exposure:  23.57 min | Livetime %:  0.13
12-apr-19
Duration:  273.77 min | Effective Exposure:  3.14 min | Livetime %:  0.01
13-apr-19
Duration:  355.88 min | Effective Exposure:  2.99 min | Liv

In [ ]:
import pickle
import numpy as np

aia_file = '/Users/jmdunca2/do-dem/initial_dem_6jun20/all_aia_dicts_06-jun-20_post/orbit_20611003001/20-59-25_20-59-55_aia_prep.pickle'
with open(aia_file, 'rb') as f:
    aia_dat = pickle.load(f)

np.isnan(aia_dat['aia_dn_s_px']).any()

In [ ]:
done_sources = {k: all_targets[k] for k in keys}
ana.get_exposures(done_sources, dogoes=False)

In [ ]:
import pandas as pd

df = pd.read_csv('fpmA.csv')
starts = df['flare_start'].values
stops = df['flare_end'].values

for i in range(0, len(starts)-1):
    print(starts[i], stops[i])

In [ ]:
for f in flaretimes:
    print(f[0], f[1])

In [ ]:
flaretimes=[]
nonflaretimes=[]
for time in all_time_intervals_list:
    b4 = [s < time[0] for s in starts]
    ea = [s > time[0] for s in stops]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        #print(starts[es], stops[es])
        #print(time)
        flaretimes.append(time)
        continue

    b4 = [s > time[0] for s in starts]
    ea = [s < time[1] for s in starts]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        #print(starts[es], stops[es])
        #print(time)
        flaretimes.append(time)
        continue

    nonflaretimes.append(time)

For the time interval to overlap with a flare, it could either ENVELOP a flare, start before (and end during) start during (and end during) and start + end within the flare. 

- fstart tstart tend fend - ENVELOP
- fstart tstart fend tend - STOPSIDE
  
- tstart fstart tend fend - STARTSIDE
- tstart fstart fend tend - DURING



In [ ]:
from matplotlib import pyplot as plt 

logbins = np.geomspace(np.min(all_above10s), np.max(all_above10s), 30)

fig, ax = plt.subplots(figsize=(15,4), tight_layout = {'pad': 1})

ax.hist(all_above10s, bins=logbins, color='skyblue', edgecolor='black')
ax.set_xscale('log')
ax.axvline(1.8e22, color='Red')
ax.axvline(1.5e23, color='Red')
ax.axvspan(1.8e22, 1.5e23, alpha=0.3, color='Red', label='Ishikawa (2017) 95% Interval')
ax.set_ylabel('Number of intervals')
ax.set_xlabel('EM Integrated >10 MK')
ax.legend()

In [ ]:
# import glob

# #Set path to obsid directory - initial pipeline should have been run already.
# ind=0
# datapath=id_dirs[ind]
# obsid=obsids[ind]

# evt_data, hdr = nu.return_submap(datapath=datapath, fpm='A', return_evt_hdr=True)
# time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
# timerange = [time0.tt.datetime, time1.tt.datetime]
# from datetime import timezone
# timerange = [t.replace(tzinfo=timezone.utc) for t in timerange]

# #Comment second line if you're not using this same example nustar orbit
# #Edit it to include only the desired time interval (default- all times in file) once you've run this once
# #timerange=[]
# #timerange=[datetime.datetime(2018, 5, 29, 22, 22), datetime.datetime(2018, 5, 29, 23, 20)]

# evtA = glob.glob(datapath+'/event_cl/*A06_cl.evt')
# evtB = glob.glob(datapath+'/event_cl/*B06_cl.evt')
# hkA  = glob.glob(datapath+'/hk/*A_fpm.hk')
# hkB  = glob.glob(datapath+'/hk/*B_fpm.hk')

# import lightcurves as lc

# importlib.reload(lc)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[2.,4.], 
#                               livetime_corr=False, save_dir=working_dir)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[4.,6.], 
#                               livetime_corr=False, save_dir=working_dir)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[6.,10.], 
#                               livetime_corr=False, save_dir=working_dir)

# lc.plot_nustar_lightcurves(eranges = [[2.,4.],[4.,6.],[6.,10.]],
#                            timerange=timerange, save_dir=working_dir)